# [초급 프로젝트] 4팀_김명환

---
---

# 프로그래밍

In [ ]:
!pip install -q gdown
!pip install -q albumentations
!pip install -q ultralytics
!pip install -q -U ultralytics
!pip install -q nbformat
!pip install -q roboflow
!pip install -q opencv-python
!pip install -q opencv-python-headless
!pip install -q wandb
!pip install -q timm
!pip install -q torchvision
#!pip install -q torch torchvision tqdm pillow matplotlib

print("로딩완료")

In [ ]:
!wandb login 86a7b8c07184b2efdfb116546a17b1905e41cb5d

In [ ]:
# 기본 라이브러리 (중복 제거 및 정리)

# --- Scikit-learn: 데이터 전처리, 모델, 평가 ---
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.datasets import (
    fetch_california_housing, load_iris, make_moons, make_circles,
    load_breast_cancer, load_wine
)
from sklearn import datasets
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, mean_squared_error, average_precision_score

# --- 이미지 처리 ---
import cv2
from PIL import Image, ImageFilter, ImageDraw
import albumentations as A

# --- PyTorch: 딥러닝 관련 ---
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader, Subset
# 문제 있는 v2 import 제거하고 필요시에만 개별적으로 import
# from torchvision.transforms import v2, functional as TF
from torchvision.transforms import functional as TF
from torchvision.datasets import CocoDetection
from torch.nn import CrossEntropyLoss
from collections import OrderedDict

# --- COCO 데이터셋 관련 ---
from pycocotools.coco import COCO
from pycocotools import mask as coco_mask

# --- 딥러닝 모델 ---
import timm

# --- 기본 라이브러리 ---
import os
import sys
import re
import csv
import copy
import json
import math
import random
import yaml
import shutil
import requests
import xml.etree.ElementTree as ET
from pathlib import Path

# --- 데이터 분석 및 시각화 ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# --- 시간 관련 ---
from datetime import datetime, timezone, timedelta
import pytz

# --- 진행률 표시 ---
import IPython.display
from tqdm.notebook import tqdm

# --- 시간대 설정 ---
__kst = pytz.timezone('Asia/Seoul')

# --- GPU 설정 ---
__device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
__device_cpu = torch.device('cpu')

# --- 재현 가능한 결과를 위한 시드 설정 ---
np.random.seed(42)
torch.manual_seed(42)
if __device.type == 'cuda':
    torch.cuda.manual_seed_all(42)

print(f"라이브러리 로드 완료 사용장치: {__device}")

In [ ]:
from urllib.request import urlretrieve; urlretrieve("https://raw.githubusercontent.com/c0z0c/jupyter_hangul/refs/heads/beta/helper_c0z0c_dev.py", "helper_c0z0c_dev.py")
import importlib
import helper_c0z0c_dev as helper
importlib.reload(helper)

In [ ]:
import os, sys
from pathlib import Path

utils_dir = None
if helper.is_colab:
    utils_dir = "/content/drive/MyDrive/codeit_ai_health_eat/src/python_modules/utils"
else:
    utils_dir = os.path.join(Path.cwd().drive + '\\', 'GoogleDrive', "codeit_ai_health_eat", "src", "python_modules", "utils")

print("utils_dir:", utils_dir)

sys.path.append(str(utils_dir))
print("sys.path:", sys.path)
import importlib
import health_ea_utils as heu
importlib.reload(heu)
from health_ea_utils import *

print("helper.__file__:", helper.__file__)
print("health_ea_utils.__file__:", heu.__file__)


# 1. 학습용 데이타 다운로드 및 압축 풀기

In [ ]:
def get_tqdm_kwargs():
    """Widget 오류를 방지하는 안전한 tqdm 설정"""
    return {
        'disable': False,
        'leave': True,
        'file': sys.stdout,
        'ascii': True,  # ASCII 문자만 사용
        'dynamic_ncols': False,
#        'ncols': 80  # 고정 폭
    }

def drive_root():
    root_path = os.path.join("D:\\", "GoogleDrive")
    if helper.is_colab:
        root_path = os.path.join("/content/drive/MyDrive")
    return root_path

def get_path_modeling(add_path = None):
    modeling_path = "modeling_yolo"
    path = os.path.join(drive_root(),modeling_path)
    if add_path is not None:
        path = os.path.join(path,add_path)
    return path

def get_path_modeling_release(add_path = None):
    modeling_path = "modeling_yolo"
    path = os.path.join(drive_root(),modeling_path)
    if add_path is not None:
        path = os.path.join(path,add_path)
    return path

def print_dir_tree(root, max_depth=2, list_count=3, indent=""):
    import os
    if max_depth < 0:
        return
    try:
        items = os.listdir(root)
    except Exception as e:
        print(indent + f"[Error] {e}")
        return

    img_count = len([f for f in os.listdir(root) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.xml', '.inf', '.txt'))])
    for item in items:
        path = os.path.join(root, item)
        if os.path.isdir(path):
            print(indent + "|-- "+ item)
            # 이미지 파일 개수만 출력
            img_count = len([f for f in os.listdir(path) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.xml', '.inf', '.txt'))])
            if img_count > list_count:
                print(indent + "   "+ f"[데이터파일: {img_count}개]")
            print_dir_tree(root=path, max_depth=max_depth-1, list_count=list_count, indent=indent + "   ")
        else:
            if list_count < img_count and item.lower().endswith(('.jpg', '.jpeg', '.png', '.xml', '.inf', '.txt')):
                continue
            print(indent + "|-- "+ item)

def save_model_dict(model, path, pth_name, kwargs=None):
    """모델 state_dict와 추가 정보를 저장"""
    def safe_makedirs(path):
        """안전한 디렉토리 생성"""
        if os.path.exists(path) and not os.path.isdir(path):
            os.remove(path)  # 파일이면 삭제
        os.makedirs(path, exist_ok=True)

    # 디렉토리 생성
    safe_makedirs(path)

    # 모델 구조 정보 추출
    model_info = {
        'class_name': model.__class__.__name__,
        'init_args': {},
        'str': str(model),
        'repr': repr(model),
        'modules': [m.__class__.__name__ for m in model.modules()],
    }

    # 생성자 인자 자동 추출(가능한 경우)
    if hasattr(model, '__dict__'):
        for key in ['in_ch', 'base_ch', 'num_classes', 'out_ch']:
            if hasattr(model, key):
                model_info['init_args'][key] = getattr(model, key)

    # kwargs 처리
    extra_info = {}
    if kwargs is not None:
        if isinstance(kwargs, str):
            extra_info = json.loads(kwargs)
        elif isinstance(kwargs, dict):
            extra_info = kwargs

    model_info.update(extra_info)

    # 저장할 dict 구성
    save_dict = {
        'model_state': model.state_dict(),
        'class_name': model.__class__.__name__,
        'model_info': model_info,
    }

    save_path = os.path.join(path, f"{pth_name}.pth")
    torch.save(save_dict, save_path)
    return save_path

def load_model_dict(path, pth_name=None):
    """
    save_model_dict로 저장한 모델을 불러오는 함수
    반환값: (model_state, model_info)
    """
    import torch
    load_path = path
    if pth_name is not None:
        load_path = os.path.join(path, f"{pth_name}.pth")
    checkpoint = torch.load(load_path, map_location='cpu', weights_only=False)  # <-- 여기 추가
    model_state = checkpoint.get('model_state')
    model_info = checkpoint.get('model_info')
    model_info['file_name'] = os.path.basename(load_path)
    return model_state, model_info


def search_pth_files(base_path):
    """
    입력된 경로의 하위 폴더들에서 pth 파일들을 검색
    """
    pth_files = []

    if not os.path.exists(base_path):
        print(f"경로가 존재하지 않습니다: {base_path}")
        return pth_files

    print(f"pth 파일 검색 시작: {base_path}")

    # 하위 폴더들을 순회하며 pth 파일 검색
    for root, dirs, files in os.walk(base_path):
        for file in files:
            if file.endswith('.pth'):
                pth_path = os.path.join(root, file)
                pth_files.append(pth_path)

    # 결과 정리 및 출력
    if pth_files:
        print(f"\n발견된 pth 파일들 ({len(pth_files)}개):")
        for i, pth_file in enumerate(pth_files, 1):
            # 상대 경로로 표시 (base_path 기준)
            rel_path = os.path.relpath(pth_file, base_path)
            print(f" {i:2d}. {rel_path}")
    else:
        print("pth 파일을 찾을 수 없습니다.")

    return pth_files

def print_json_tree(data, indent="", max_depth=4, _depth=0, list_count=2, print_value=True):
    """
    JSON 객체를 지정한 단계(max_depth)까지 트리 형태로 출력
    - list 타입은 3개 이상일 때 개수만 출력
    - 하위 노드가 값일 경우 key(type) 형태로 출력
    - print_value=True일 때 key(type): 값 형태로 출력
    """
    if _depth > max_depth:
        return
    if isinstance(data, dict):
        for key, value in data.items():
            if isinstance(value, (dict, list)):
                print(f"{indent}|-- {key}")
                print_json_tree(value, indent + "    ", max_depth, _depth + 1, list_count, print_value)
            else:
                if print_value:
                    print(f"{indent}|-- {key}({type(value).__name__}): {value if len(str(value)) < 100 else f'{str(value)[:30]}...'}")
                else:
                    print(f"{indent}|-- {key}({type(value).__name__})")
    elif isinstance(data, list):
        if len(data) > list_count:
            print(f"{indent}|-- [list] ({len(data)} items)")
        else:
            for i, item in enumerate(data):
                if isinstance(item, (dict, list)):
                    print(f"{indent}|-- [{i}]")
                    print_json_tree(item, indent + "    ", max_depth, _depth + 1, list_count, print_value)
                else:
                    if print_value:
                        print(f"{indent}|-- [{i}]({type(item).__name__}): {item if len(str(item)) < 100 else f'{str(item)[:30]}...'}")
                    else:
                        print(f"{indent}|-- [{i}]({type(item).__name__})")
    else:
        if print_value:
            print(f"{indent}{type(data).__name__}: {data if len(str(data)) < 100 else f'{str(data)[:30]}...'}")
        else:
            print(f"{indent}{type(data).__name__}")

def print_git_tree(data, indent="", max_depth=3, _depth=0):
    """
    PyTorch tensor/딕셔너리/리스트를 git tree 스타일로 출력
    """
    import torch
    import numpy as np

    if _depth > max_depth:
        return
    if isinstance(data, dict):
        for key, value in data.items():
            print(f"{indent}├─ {key} [{type(value).__name__}]")
            print_git_tree(value, indent + "│  ", max_depth, _depth + 1)
    elif isinstance(data, (list, tuple)):
        for i, item in enumerate(data):
            print(f"{indent}├─ [{i}] [{type(item).__name__}]")
            print_git_tree(item, indent + "│  ", max_depth, _depth + 1)
    elif torch.is_tensor(data):
        shape = tuple(data.shape)
        dtype = str(data.dtype)
        preview = str(data)
        preview_str = preview[:80] + ("..." if len(preview) > 80 else "")
        print(f"{indent}└─ Tensor shape={shape} dtype={dtype} preview={preview_str}")
    elif isinstance(data, np.ndarray):
        shape = data.shape
        dtype = data.dtype
        preview = str(data)
        preview_str = preview[:80] + ("..." if len(preview) > 80 else "")
        print(f"{indent}└─ ndarray shape={shape} dtype={dtype} preview={preview_str}")
    else:
        val_str = str(data)
        print(f"{indent}└─ {type(data).__name__}: {val_str[:80]}{'...' if len(val_str)>80 else ''}")


print("유틸리티 함수 로드 완료")

# 데이타 다운로드

In [ ]:
# download_files={
#     'yolo_label_one_class' : r'https://drive.google.com/file/d/177_86k4BuT6JnFnq7ZHJtEjp7jaRbCl2/view?usp=sharing',
#     'yolo_label' : r'https://drive.google.com/file/d/1nc-WFcw7lCS7s7VGzN9Kxh80PiBBggez/view?usp=sharing',
#     'yolo_resize_one_class' : r'https://drive.google.com/file/d/1Ak0EvkMnuwvcAFvTO-zovIgVcNlROjsS/view?usp=sharing',
#     'yolo_resize' : r'https://drive.google.com/file/d/1kpo57qOJhEhrkuzUCEh57ILB5xSPVoFv/view?usp=sharing',
# }

# download_files={
#     'yolo_label' : r'https://fs.mybox.naver.com/file/download.api?resourceKey=YzB6MGN8MzQ3MjU5Nzc1ODU5OTkzNTMyOHxGfDA&svcType=MYBOX-WEB&time=1757776010785',
#     'yolo_label_one_class' : r'https://fs.mybox.naver.com/file/download.api?resourceKey=YzB6MGN8MzQ3MjU5Nzc1ODYzOTg5NDExMnxGfDA&svcType=MYBOX-WEB&time=1757776673721',
#     'yolo_resize_one_class' : r'https://fs.mybox.naver.com/file/download.api?resourceKey=YzB6MGN8MzQ3MjU5Nzc1ODgwNjk2NDMyMHxGfDA&svcType=MYBOX-WEB&time=1757780142635',
#     'yolo_resize' : r'https://fs.mybox.naver.com/file/download.api?resourceKey=YzB6MGN8MzQ3MjU5Nzc1ODY4MDc2MjQ2NHxGfDA&svcType=MYBOX-WEB&time=1757780177672',
# }

download_files={
    # 'yolo_label' : r'https://fs.mybox.naver.com/file/download.api?resourceKey=YzB6MGN8MzQ3MjU5Nzc1ODU5OTkzNTMyOHxGfDA&svcType=MYBOX-WEB&time=1757776010785',
    # 'yolo_label_one_class' : r'https://fs.mybox.naver.com/file/download.api?resourceKey=YzB6MGN8MzQ3MjU5Nzc1ODYzOTg5NDExMnxGfDA&svcType=MYBOX-WEB&time=1757776673721',
    # 'yolo_resize_one_class' : r'https://fs.mybox.naver.com/file/download.api?resourceKey=YzB6MGN8MzQ3MjU5Nzc1ODgwNjk2NDMyMHxGfDA&svcType=MYBOX-WEB&time=1757780142635',
    # 'yolo_resize' : r'https://fs.mybox.naver.com/file/download.api?resourceKey=YzB6MGN8MzQ3MjU5Nzc1ODY4MDc2MjQ2NHxGfDA&svcType=MYBOX-WEB&time=1757780177672',
    'yolo_noresize' : r'https://fs.mybox.naver.com/file/download.api?resourceKey=YzB6MGN8MzQ3MjU5Nzc2NDA0ODY2ODI1NnxGfDA&svcType=MYBOX-WEB&time=1757851996107',
}
# yolo_noresize = https://fs.mybox.naver.com/file/download.api?resourceKey=YzB6MGN8MzQ3MjU5Nzc2NDA0ODY2ODI1NnxGfDA&svcType=MYBOX-WEB&time=1757851996107




In [ ]:
import gdown
def download_gdrive_file(url, output_path, ignore=True):
    # 공유 링크에서 파일 ID 추출
    if os.path.exists(output_path):
        if ignore:
            os.remove(output_path)
        else:
            return

    file_id_match = re.search(r'/d/([a-zA-Z0-9_-]+)', url)
    if not file_id_match:
        raise ValueError("Google Drive 파일 ID를 찾을 수 없습니다.")
    file_id = file_id_match.group(1)
    gdown.download(f"https://drive.google.com/uc?id={file_id}", output_path, quiet=False)

def download_http(url, target, ignore=True):
    """
    HTTP 파일 다운로드 함수 (진행률 표시)
    url: 다운로드할 파일 URL
    target: 저장할 파일 경로
    ignore: True면 기존 파일 삭제 후 다운로드, False면 파일 있으면 건너뜀
    """
    if os.path.exists(target):
        if ignore:
            os.remove(target)
        else:
            print(f"이미 파일이 존재합니다: {target}")
            return target

    response = requests.get(url, stream=True)
    total = int(response.headers.get('content-length', 0))
    with open(target, 'wb') as file, tqdm(
        desc=f"Downloading {os.path.basename(target)}",
        total=total,
        unit='B',
        unit_scale=True,
        unit_divisor=1024,
        ascii=True
    ) as bar:
        for data in response.iter_content(chunk_size=1024):
            size = file.write(data)
            bar.update(size)
    print(f"다운로드 완료: {target}")
    return target

# local_code_it_ai04 = os.path.join( '~/.cache/' if helper.is_colab else Path.cwd().drive + '\\'
#                                   ,'temp'
#                                   , 'code_it_ai04')

if helper.is_colab:
    local_code_it_ai04 = os.path.join( '/content/', 'code_it_ai04')
else:
    local_code_it_ai04 = os.path.join( Path.cwd().drive + '\\', 'temp', 'code_it_ai04')

print("local_code_it_ai04:", local_code_it_ai04)

os.makedirs(local_code_it_ai04, exist_ok=True)  # 폴더 생성 코드 추가
unzip_paths = []
for key, url in download_files.items():
    print(f"{key}: {url}")
    zipfile = os.path.join(local_code_it_ai04, f'{key}.zip')
    unzip_path = os.path.join(local_code_it_ai04, f'{key}.zip.unzip')
    if os.path.exists(unzip_path):
        print(f"이미 압축해제된 폴더가 존재합니다: {unzip_path}")
        print('unzipfile:', unzip_path)
        unzip_paths.append(unzip_path)
        continue
    #download_gdrive_file(url, os.path.join(local_code_it_ai04, f'{key}.zip'), ignore=False)
    download_http(url, zipfile, ignore=False)
    unzip_path_list = heu.unzip([os.path.join(local_code_it_ai04, f'{key}.zip')])
    # for p in unzip_path_list:
    #     unzip_paths.append(p)
    print('unzip_path_list:', unzip_path_list)
    unzip_paths.extend(unzip_path_list)


### > 설정 < 플레그

In [ ]:
# google drive root에 keggle.json 파일 필요합니다.
for path in unzip_paths:
    print("압축해제된 폴더:", path)

#yolo_dataset_path = os.path.join(local_code_it_ai04, f'yolo_label_one_class.zip.unzip')
yolo_dataset_path =unzip_paths[0]
yaml_path = os.path.join(yolo_dataset_path, "dataset.yaml")

def get_path_data():
    path = yolo_dataset_path
    return path

print("yaml_path:", yaml_path)
print("get_path_data:", get_path_data())

## EfficientNet 모델링

In [ ]:
for path in unzip_paths:
    print(f"압축해제된 폴더: {path}")


In [ ]:
class YOLOToClassificationDataset(Dataset):
    """YOLO 형식 데이터를 분류용으로 변환하는 데이터셋"""

    def __init__(self, yaml_path, split='train', transform=None, crop_objects=True):
        # yaml 파일 읽기
        with open(yaml_path, 'r') as f:
            yaml_data = yaml.safe_load(f)

        self.yaml_data_path = yaml_data['path']
        self.nc = yaml_data['nc']
        self.names = yaml_data['names']
        self.yaml_data_train = os.path.join(yaml_data['path'], yaml_data['train'])
        self.yaml_data_val = os.path.join(yaml_data['path'], yaml_data['val'])
        self.yaml_data_test = os.path.join(yaml_data['path'], yaml_data['test'])

        #print("nc:", self.nc)
        #print("names:", self.names)
        print("yaml_data_path:", self.yaml_data_path)
        #print("yaml_data_train:", self.yaml_data_train)
        #print("yaml_data_val:", self.yaml_data_val)
        #print("yaml_data_test:", self.yaml_data_test)

        # split에 따라 경로 선택
        if split == 'train':
            image_dir = self.yaml_data_train
        elif split == 'val':
            image_dir = self.yaml_data_val
        elif split == 'test':
            image_dir = self.yaml_data_test
        else:
            raise ValueError(f"split 값이 잘못되었습니다: {split}")

        label_dir = image_dir.replace('images', 'labels')
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.class_names = self.names
        self.transform = transform
        self.crop_objects = crop_objects
        self.data = []

        self._load_data()

    def _load_data(self):
        if not os.path.exists(self.label_dir):
            #print(f"라벨 폴더가 존재하지 않습니다: {self.label_dir}")
            return  # 라벨 폴더 없으면 빈 데이터셋
        for label_file in os.listdir(self.label_dir):
            if not label_file.endswith('.txt'):
                continue
            image_file = label_file.replace('.txt', '.jpg')
            image_path = os.path.join(self.image_dir, image_file)
            label_path = os.path.join(self.label_dir, label_file)
            if not os.path.exists(image_path):
                continue
            with open(label_path, 'r') as f:
                lines = f.readlines()
            for line in lines:
                parts = line.strip().split()
                if len(parts) >= 5:
                    class_id = int(parts[0])
                    x_center, y_center, width, height = map(float, parts[1:5])
                    self.data.append({
                        'image_path': image_path,
                        'class_id': class_id,
                        'bbox': (x_center, y_center, width, height)
                    })

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        image = Image.open(item['image_path']).convert('RGB')
        if self.crop_objects and 'bbox' in item:
            img_w, img_h = image.size
            x_center, y_center, width, height = item['bbox']
            x1 = int((x_center - width/2) * img_w)
            y1 = int((y_center - height/2) * img_h)
            x2 = int((x_center + width/2) * img_w)
            y2 = int((y_center + height/2) * img_h)
            # 좌표가 올바른지 체크
            if x2 > x1 and y2 > y1 and x1 >= 0 and y1 >= 0 and x2 <= img_w and y2 <= img_h:
                image = image.crop((x1, y1, x2, y2))
            # else: 잘못된 bbox는 crop하지 않음
        if self.transform:
            image = self.transform(image)
        return image, item['class_id']
    def get_item(self, idx):
        item = self.data[idx]
        image = Image.open(item['image_path']).convert('RGB')
        if self.crop_objects and 'bbox' in item:
            img_w, img_h = image.size
            x_center, y_center, width, height = item['bbox']
            x1 = int((x_center - width/2) * img_w)
            y1 = int((y_center - height/2) * img_h)
            x2 = int((x_center + width/2) * img_w)
            y2 = int((y_center + height/2) * img_h)
            # 좌표가 올바른지 체크
            if x2 > x1 and y2 > y1 and x1 >= 0 and y1 >= 0 and x2 <= img_w and y2 <= img_h:
                image = image.crop((x1, y1, x2, y2))
            # else: 잘못된 bbox는 crop하지 않음
        return image, item['class_id'], item['image_path'], [x1, y1, x2, y2]


In [ ]:

# 데이터 전처리 파이프라인
def get_transforms():
    train_transform = transforms.Compose([
        transforms.Resize((300, 300)),  # EfficientNet-B3 적절한 크기
        transforms.RandomHorizontalFlip(0.5),
        transforms.RandomRotation(10),
        transforms.ColorJitter(brightness=0.2, contrast=0.2),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                           std=[0.229, 0.224, 0.225])
    ])

    val_transform = transforms.Compose([
        transforms.Resize((300, 300)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                           std=[0.229, 0.224, 0.225])
    ])

    return train_transform, val_transform

# EfficientNet-B3 모델 생성
def create_efficientnet_model(num_classes, pretrained=True):
    """사전 학습된 EfficientNet-B3 모델 로드"""
    model = timm.create_model('efficientnet_b3',
                             pretrained=pretrained,
                             num_classes=num_classes)
    return model

# 학습 함수
def train_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    pbar = tqdm(dataloader, desc='Training')
    for images, labels in pbar:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        pbar.set_postfix({
            'Loss': f'{running_loss/len(dataloader):.4f}',
            'Acc': f'{100*correct/total:.2f}%'
        })

    return running_loss/len(dataloader), 100*correct/total

# 검증 함수
def validate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in tqdm(dataloader, desc='Validating'):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return running_loss/len(dataloader), 100*correct/total


In [ ]:
def update_yaml_paths_to_absolute(yaml_path):
    with open(yaml_path, 'r') as f:
        data = yaml.safe_load(f)

    yaml_dir = os.path.dirname(yaml_path)
    data['path'] = os.path.normpath(os.path.join(yaml_dir, data['path']))
    # for key in ['train', 'val', 'test']:
    #     if key in data and not os.path.isabs(data[key]):
    #         data[key] = os.path.normpath(os.path.join(yaml_dir, data[key]))

    with open(yaml_path, 'w') as f:
        yaml.dump(data, f, allow_unicode=True)

    return data

yaml_data = update_yaml_paths_to_absolute(yaml_path)
print(yaml_data.keys())
yaml_data_path = yaml_data['path']
nc = yaml_data['nc']
names = yaml_data['names']
yaml_data_train = os.path.join(yaml_data['path'], yaml_data['train'])
yaml_data_val = os.path.join(yaml_data['path'], yaml_data['val'])
yaml_data_test = os.path.join(yaml_data['path'], yaml_data['test'])

print("nc:", nc)
print("names:", names)
print("yaml_data_path:", yaml_data_path)
print("yaml_data_train:", yaml_data_train)
print("yaml_data_val:", yaml_data_val)
print("yaml_data_test:", yaml_data_test)


In [ ]:
# raise ValueError("중단")

In [ ]:
# sample image
def save_sample(dataset, save_dir, type, num_samples=5):
    os.makedirs(save_dir, exist_ok=True)
    pbar = tqdm(range(min(num_samples, len(dataset))), desc="샘플 이미지 저장 중")
    for i in pbar:
        image, label, class_image_path, bbox = dataset.get_item(i)
        base_file_name = os.path.basename(class_image_path).split('.')[0]
        class_name = dataset.class_names[label]
        image_path = os.path.join(save_dir, 'images', type, f'{base_file_name}_{class_name}.jpg')
        os.makedirs(os.path.dirname(image_path), exist_ok=True)  # 폴더 생성 추가
        # 텐서를 PIL 이미지로 변환
        if isinstance(image, torch.Tensor):
            image = TF.to_pil_image(image)
        image.save(image_path)
        label_path = os.path.join(save_dir, 'labels', type, f'{base_file_name}_{class_name}.txt')
        os.makedirs(os.path.dirname(label_path), exist_ok=True)  # 폴더 생성 추가
        with open(label_path, 'w') as f:
            f.write(f"{label}\n")
        pbar.set_postfix_str(f'{base_file_name}_{class_name}')

def save_classification_yaml(save_dir, class_names):
    yaml_dict = {
        'path': '.',
        'train': 'images/train',
        'val': 'images/val',
        'test': 'images/test',
        'nc': len(class_names),
        'names': class_names
    }
    yaml_path = os.path.join(save_dir, 'dataset.yaml')
    with open(yaml_path, 'w', encoding='utf-8') as f:
        yaml.dump(yaml_dict, f, allow_unicode=True)
    print(f'dataset.yaml 저장: {yaml_path}')

def save_yaml_sample_images():
    yaml_data = update_yaml_paths_to_absolute(yaml_path)
    print(yaml_data.keys())
    yaml_data_path = yaml_data['path']
    nc = yaml_data['nc']
    names = yaml_data['names']
    yaml_data_train = os.path.join(yaml_data['path'], yaml_data['train'])
    yaml_data_val = os.path.join(yaml_data['path'], yaml_data['val'])
    yaml_data_test = os.path.join(yaml_data['path'], yaml_data['test'])


    CLASS_NAMES = names
    NUM_CLASSES = len(CLASS_NAMES)
    BATCH_SIZE = 16
    LEARNING_RATE = 0.001
    NUM_EPOCHS = 10
    # ========================================

    # 데이터셋 및 데이터로더 생성
    train_transform, val_transform = get_transforms()

    # 전체 데이터셋 (실제로는 train/val 분할 필요)
    train_dataset = YOLOToClassificationDataset(
        yaml_path = yaml_path,
        split='train',
        transform=train_transform
    )
    val_dataset = YOLOToClassificationDataset(
        yaml_path = yaml_path,
        split='val',
        transform=val_transform
    )
    test_dataset = YOLOToClassificationDataset(
        yaml_path = yaml_path,
        split='test',
        transform=val_transform
    )
    yaml_data_sample_images_train_path = os.path.join(f"{yaml_data_path}_one")
    yaml_data_sample_images_val_path = os.path.join(f"{yaml_data_path}_one")
    save_sample(train_dataset, yaml_data_sample_images_train_path, 'train', num_samples=len(train_dataset))
    save_sample(val_dataset, yaml_data_sample_images_val_path, 'val', num_samples=len(val_dataset))
    save_classification_yaml(yaml_data_sample_images_train_path, CLASS_NAMES)
    print("샘플 이미지 저장 완료")

# 알약 별로 이미지를 분리한 후 저장
# save_yaml_sample_images()

In [ ]:
# raise ValueError("중단")

In [ ]:

# 메인 실행 함수
def main_efficientnet():
    # ========== 설정 부분 (수정 필요) ==========
    yaml_data = update_yaml_paths_to_absolute(yaml_path)
    print(yaml_data.keys())
    yaml_data_path = yaml_data['path']
    nc = yaml_data['nc']
    names = yaml_data['names']
    yaml_data_train = os.path.join(yaml_data['path'], yaml_data['train'])
    yaml_data_val = os.path.join(yaml_data['path'], yaml_data['val'])
    yaml_data_test = os.path.join(yaml_data['path'], yaml_data['test'])

    CLASS_NAMES = names
    NUM_CLASSES = len(CLASS_NAMES)
    BATCH_SIZE = 16
    LEARNING_RATE = 0.001
    NUM_EPOCHS = 10

    # 모델 저장 경로 설정 (YOLO와 동일한 구조)
    model_save_name = f"efficientnet_b3_cls_{NUM_CLASSES}classes"
    model_save_dir = get_path_modeling()
    model_project_dir = os.path.join(model_save_dir, model_save_name)
    os.makedirs(model_project_dir, exist_ok=True)

    print(f"모델 저장 경로: {model_project_dir}")
    # ========================================

    # 데이터셋 및 데이터로더 생성
    train_transform, val_transform = get_transforms()

    train_dataset = YOLOToClassificationDataset(
        yaml_path = yaml_path,
        split='train',
        transform=train_transform
    )
    val_dataset = YOLOToClassificationDataset(
        yaml_path = yaml_path,
        split='val',
        transform=val_transform
    )
    test_dataset = YOLOToClassificationDataset(
        yaml_path = yaml_path,
        split='test',
        transform=val_transform
    )

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

    print(f"Training samples: {len(train_dataset)}")
    print(f"Validation samples: {len(val_dataset)}")
    print(f"Test samples: {len(test_dataset)}")

    # 모델 생성
    model = create_efficientnet_model(NUM_CLASSES, pretrained=True)
    model = model.to(__device)

    # 손실 함수와 옵티마이저
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

    # 학습 루프
    best_val_acc = 0.0
    train_losses, train_accs = [], []
    val_losses, val_accs = [], []

    for epoch in range(NUM_EPOCHS):
        print(f"\nEpoch {epoch+1}/{NUM_EPOCHS}")
        print("-" * 50)

        # 학습
        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, __device)
        train_losses.append(train_loss)
        train_accs.append(train_acc)

        # 검증
        val_loss, val_acc = validate(model, val_loader, criterion, __device)
        val_losses.append(val_loss)
        val_accs.append(val_acc)

        scheduler.step()

        print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
        print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")

        # 최고 성능 모델 저장 (YOLO 스타일로 수정)
        if val_acc > best_val_acc:
            best_val_acc = val_acc

            # 모델 정보 구성
            model_info = {
                'model_name': 'efficientnet_b3',
                'num_classes': NUM_CLASSES,
                'class_names': CLASS_NAMES,
                'best_val_acc': val_acc,
                'epoch': epoch + 1,
                'learning_rate': LEARNING_RATE,
                'batch_size': BATCH_SIZE,
                'train_samples': len(train_dataset),
                'val_samples': len(val_dataset),
                'test_samples': len(test_dataset),
            }

            # 베스트 모델 저장
            best_model_path = save_model_dict(
                model=model,
                path=model_project_dir,
                pth_name='best',
                kwargs=model_info
            )

            print(f"Best model saved! Val Acc: {val_acc:.2f}%")
            print(f"저장 경로: {best_model_path}")

    # 최종 모델도 저장
    final_model_info = model_info.copy()
    final_model_info.update({
        'final_val_acc': val_acc,
        'final_epoch': NUM_EPOCHS,
        'training_completed': True
    })

    final_model_path = save_model_dict(
        model=model,
        path=model_project_dir,
        pth_name='final',
        kwargs=final_model_info
    )

    print(f"Final model saved: {final_model_path}")

    # 결과 시각화 및 저장
    plt.figure(figsize=(12, 4))

    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Val Loss')
    plt.title('Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(train_accs, label='Train Acc')
    plt.plot(val_accs, label='Val Acc')
    plt.title('Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.legend()

    plt.tight_layout()

    # 차트도 같은 폴더에 저장
    chart_path = os.path.join(model_project_dir, 'training_results.png')
    plt.savefig(chart_path, dpi=150, bbox_inches='tight')
    print(f"Training chart saved: {chart_path}")

    plt.show()

    return model_project_dir, best_val_acc

In [ ]:
import wandb

def main_efficientnet_with_wandb(project_name="codeit-ai-04-01", run_name=None):
    # ========== 설정 부분 (수정 필요) ==========
    yaml_data = update_yaml_paths_to_absolute(yaml_path)
    print(yaml_data.keys())
    yaml_data_path = yaml_data['path']
    nc = yaml_data['nc']
    names = yaml_data['names']
    yaml_data_train = os.path.join(yaml_data['path'], yaml_data['train'])
    yaml_data_val = os.path.join(yaml_data['path'], yaml_data['val'])
    yaml_data_test = os.path.join(yaml_data['path'], yaml_data['test'])

    CLASS_NAMES = names
    NUM_CLASSES = len(CLASS_NAMES)
    BATCH_SIZE = 16
    LEARNING_RATE = 0.001
    NUM_EPOCHS = 10

    # 모델 저장 경로 설정 (YOLO와 동일한 구조)
    model_save_name = f"efficientnet_b3_cls_{NUM_CLASSES}classes"
    model_save_dir = get_path_modeling()
    model_project_dir = os.path.join(model_save_dir, model_save_name)
    os.makedirs(model_project_dir, exist_ok=True)

    print(f"모델 저장 경로: {model_project_dir}")

    # wandb 초기화
    wandb_config = {
        'model_name': 'efficientnet_b3',
        'num_classes': NUM_CLASSES,
        'class_names': CLASS_NAMES,
        'batch_size': BATCH_SIZE,
        'learning_rate': LEARNING_RATE,
        'num_epochs': NUM_EPOCHS,
        'image_size': 300,
        'optimizer': 'Adam',
        'scheduler': 'StepLR',
        'pretrained': True
    }

    run = wandb.init(
        project=project_name,
        name=run_name or f"efficientnet_b3_{NUM_CLASSES}classes_{datetime.now().strftime('%Y%m%d_%H%M%S')}",
        config=wandb_config,
        tags=['efficientnet', 'classification', 'pytorch']
    )
    # ========================================

    # 데이터셋 및 데이터로더 생성
    train_transform, val_transform = get_transforms()

    train_dataset = YOLOToClassificationDataset(
        yaml_path = yaml_path,
        split='train',
        transform=train_transform
    )
    val_dataset = YOLOToClassificationDataset(
        yaml_path = yaml_path,
        split='val',
        transform=val_transform
    )
    test_dataset = YOLOToClassificationDataset(
        yaml_path = yaml_path,
        split='test',
        transform=val_transform
    )

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

    print(f"Training samples: {len(train_dataset)}")
    print(f"Validation samples: {len(val_dataset)}")
    print(f"Test samples: {len(test_dataset)}")

    # wandb에 데이터셋 정보 로깅
    wandb.log({
        'dataset/train_samples': len(train_dataset),
        'dataset/val_samples': len(val_dataset),
        'dataset/test_samples': len(test_dataset),
        'dataset/total_samples': len(train_dataset) + len(val_dataset) + len(test_dataset)
    })

    # 모델 생성
    model = create_efficientnet_model(NUM_CLASSES, pretrained=True)
    model = model.to(__device)

    # wandb에 모델 구조 로깅
    wandb.watch(model, log='all', log_freq=100)

    # 손실 함수와 옵티마이저
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

    # 학습 루프
    best_val_acc = 0.0
    train_losses, train_accs = [], []
    val_losses, val_accs = [], []

    for epoch in range(NUM_EPOCHS):
        print(f"\nEpoch {epoch+1}/{NUM_EPOCHS}")
        print("-" * 50)

        # 학습
        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, __device)
        train_losses.append(train_loss)
        train_accs.append(train_acc)

        # 검증
        val_loss, val_acc = validate(model, val_loader, criterion, __device)
        val_losses.append(val_loss)
        val_accs.append(val_acc)

        scheduler.step()

        print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
        print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")

        # wandb에 메트릭 로깅
        wandb.log({
            'epoch': epoch + 1,
            'train/loss': train_loss,
            'train/accuracy': train_acc,
            'val/loss': val_loss,
            'val/accuracy': val_acc,
            'learning_rate': optimizer.param_groups[0]['lr']
        }, step=epoch)

        # 최고 성능 모델 저장
        if val_acc > best_val_acc:
            best_val_acc = val_acc

            # 모델 정보 구성
            model_info = {
                'model_name': 'efficientnet_b3',
                'num_classes': NUM_CLASSES,
                'class_names': CLASS_NAMES,
                'best_val_acc': val_acc,
                'epoch': epoch + 1,
                'learning_rate': LEARNING_RATE,
                'batch_size': BATCH_SIZE,
                'train_samples': len(train_dataset),
                'val_samples': len(val_dataset),
                'test_samples': len(test_dataset),
            }

            # 베스트 모델 저장
            best_model_path = save_model_dict(
                model=model,
                path=model_project_dir,
                pth_name='best',
                kwargs=model_info
            )

            print(f"Best model saved! Val Acc: {val_acc:.2f}%")
            print(f"저장 경로: {best_model_path}")

            # wandb에 베스트 모델 저장
            wandb.log({'best_val_accuracy': val_acc, 'best_epoch': epoch + 1})

    # 최종 모델도 저장
    final_model_info = model_info.copy()
    final_model_info.update({
        'final_val_acc': val_acc,
        'final_epoch': NUM_EPOCHS,
        'training_completed': True
    })

    final_model_path = save_model_dict(
        model=model,
        path=model_project_dir,
        pth_name='final',
        kwargs=final_model_info
    )

    print(f"Final model saved: {final_model_path}")

    # 결과 시각화 및 저장
    plt.figure(figsize=(12, 4))

    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Val Loss')
    plt.title('Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(train_accs, label='Train Acc')
    plt.plot(val_accs, label='Val Acc')
    plt.title('Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.legend()

    plt.tight_layout()

    # 차트도 같은 폴더에 저장
    chart_path = os.path.join(model_project_dir, 'training_results.png')
    plt.savefig(chart_path, dpi=150, bbox_inches='tight')
    print(f"Training chart saved: {chart_path}")

    # wandb에 차트 업로드
    wandb.log({"training_results": wandb.Image(chart_path)})

    plt.show()

    # 최종 요약 메트릭
    wandb.log({
        'final/best_val_accuracy': best_val_acc,
        'final/final_val_accuracy': val_acc,
        'final/total_epochs': NUM_EPOCHS
    })

    # 모델 아티팩트 저장
    artifact = wandb.Artifact(
        name=f"efficientnet_b3_model_{run.id}",
        type="model",
        description=f"EfficientNet-B3 model trained on {NUM_CLASSES} classes"
    )
    artifact.add_file(best_model_path)
    artifact.add_file(final_model_path)
    artifact.add_file(chart_path)
    wandb.log_artifact(artifact)

    # wandb 실행 종료
    wandb.finish()

    return model_project_dir, best_val_acc

# 실행 함수
def run_efficientnet_with_wandb():
    try:
        project_dir, best_acc = main_efficientnet_with_wandb(
            project_name="codeit-ai-04-01",
            run_name=f"efficientnet_b3_experiment_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
        )
        print(f"학습 완료! 최고 검증 정확도: {best_acc:.2f}%")
        print(f"모델 저장 경로: {project_dir}")
        return project_dir, best_acc
    except Exception as e:
        print(f"학습 중 오류 발생: {e}")
        wandb.finish()
        raise

In [ ]:
# raise ValueError('stop end')

In [ ]:
#main_efficientnet()
run_efficientnet_with_wandb()

In [ ]:
def test_model(model_path, image_path:list):
    """저장된 모델을 불러와서 테스트 이미지에 대해 예측 수행"""
    model_state, model_info = load_model_dict(model_path, pth_name='best')
    print("모델 정보:", model_info)

    num_classes = model_info['num_classes']
    class_names = model_info['class_names']

    # 모델 생성 및 state_dict 로드
    model = create_efficientnet_model(num_classes, pretrained=False)
    model.load_state_dict(model_state)
    model = model.to(__device)
    model.eval()

    # 이미지 전처리
    transform = transforms.Compose([
        transforms.Resize((300, 300)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                           std=[0.229, 0.224, 0.225])
    ])

    for img_path in image_path:
        image = Image.open(img_path).convert('RGB')
        input_tensor = transform(image).unsqueeze(0).to(__device)

        with torch.no_grad():
            output = model(input_tensor)
            _, predicted = torch.max(output, 1)
            predicted_class = class_names[predicted.item()]

        plt.imshow(image)
        plt.title(f'Predicted: {predicted_class}')
        plt.axis('off')
        plt.show()
